In [1]:
import global_align as ga

In [2]:
ncbi_match_score = 1
ncbi_mismatch_score = -2
gap_open_cost = 5
ncbi_gap_extension_score = -2
scoring_mat = {
    "A": {"A": ncbi_match_score, "C": ncbi_mismatch_score, "G": ncbi_mismatch_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "C": {"A": ncbi_mismatch_score, "C": ncbi_match_score, "G": ncbi_mismatch_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "G": {"A": ncbi_mismatch_score, "C": ncbi_mismatch_score, "G": ncbi_match_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "T": {"A": ncbi_mismatch_score, "C": ncbi_mismatch_score, "G": ncbi_mismatch_score, "T": ncbi_match_score, "-": ncbi_gap_extension_score},
    "-": {"A": ncbi_gap_extension_score, "C": ncbi_gap_extension_score, "G": ncbi_gap_extension_score, "T": ncbi_gap_extension_score, "-": ncbi_match_score},
}

max_score = ga.get_max_similarity_score(scoring_mat=scoring_mat)
cost_mat = ga.transform_scoring_mat_to_cost_mat(
    scoring_mat=scoring_mat,
    max_score=max_score
)
cost_mat

{'A': {'A': 0, 'C': 3, 'G': 3, 'T': 3, '-': 3},
 'C': {'A': 3, 'C': 0, 'G': 3, 'T': 3, '-': 3},
 'G': {'A': 3, 'C': 3, 'G': 0, 'T': 3, '-': 3},
 'T': {'A': 3, 'C': 3, 'G': 3, 'T': 0, '-': 3},
 '-': {'A': 2, 'C': 2, 'G': 2, 'T': 2, '-': 0}}

In [3]:
# seq_1, seq_2 = ga.draw_two_random_seqs(
#     alphabet=["A", "C", 'G', "T"],
#     min_len_seq_1=5,
#     max_len_seq_1=10,
#     min_len_seq_2=5,
#     max_len_seq_2=10,
#     divergence=0.8
# )
seq_1 = "GTAGGCGGTC"
seq_2 = "CAGCTGC"
print(seq_1)
print(seq_2)

GTAGGCGGTC
CAGCTGC


In [4]:
middle_row_index = len(seq_1)
best_paths_mat = ga.init_best_paths_matrix(
    dynamic_prog_num_rows=middle_row_index + 1,
    dynamic_prog_num_cols=len(seq_2) + 1
)
best_paths_mat

[[2, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2],
 [4, 2, 2, 2, 2, 2, 2, 2]]

In [5]:
dynamic_prog_num_cols = len(seq_2) + 1
partial_dp_mat = ga.init_partial_dynamic_prog_matrix_2(
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost,
    dynamic_prog_num_cols=dynamic_prog_num_cols
)
partial_dp_mat

[[0, 7, 9, 11, 13, 15, 17, 19], [8, 0, 0, 0, 0, 0, 0, 0]]

In [6]:
moves_for_gap_open_penalty_from_left = {0, 3, 4, 11, 12, 14, 19, 22, 23}
moves_for_gap_open_penalty_from_up = {0, 1, 2, 9, 10, 13, 19, 20, 21}
situation_mapper={
    # from_diag_best_path_type = 0
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (0, 1, 3)): 15,
    # 2-way ties for low
    ((0, 0, 2), (0, 1, 3)): 9,
    ((0, 2, 0), (0, 1, 3)): 11,
    ((2, 0, 0), (0, 1, 3)): 5,
    # 2-way ties for high
    ((0, 1, 1), (0, 1, 3)): 0,
    ((1, 0, 1), (0, 1, 3)): 1,
    ((1, 1, 0), (0, 1, 3)): 3,
    # no ties
    ((0, 1, 2), (0, 1, 3)): 0,
    ((1, 0, 2), (0, 1, 3)): 1,
    ((1, 2, 0), (0, 1, 3)): 3,
    ((0, 2, 1), (0, 1, 3)): 0,
    ((2, 0, 1), (0, 1, 3)): 1,
    ((2, 1, 0), (0, 1, 3)): 3,
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (0, 1, 4)): 16,
    # 2-way ties for low
    ((0, 0, 2), (0, 1, 4)): 9,
    ((0, 2, 0), (0, 1, 4)): 12,
    ((2, 0, 0), (0, 1, 4)): 6,
    # 2-way ties for high
    ((0, 1, 1), (0, 1, 4)): 0,
    ((1, 0, 1), (0, 1, 4)): 1,
    ((1, 1, 0), (0, 1, 4)): 4,
    # no ties
    ((0, 1, 2), (0, 1, 4)): 0,
    ((1, 0, 2), (0, 1, 4)): 1,
    ((1, 2, 0), (0, 1, 4)): 4,
    ((0, 2, 1), (0, 1, 4)): 0,
    ((2, 0, 1), (0, 1, 4)): 1,
    ((2, 1, 0), (0, 1, 4)): 4,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (0, 2, 3)): 17,
    # 2-way ties for low
    ((0, 0, 2), (0, 2, 3)): 10,
    ((0, 2, 0), (0, 2, 3)): 11,
    ((2, 0, 0), (0, 2, 3)): 7,
    # 2-way ties for high
    ((0, 1, 1), (0, 2, 3)): 0,
    ((1, 0, 1), (0, 2, 3)): 2,
    ((1, 1, 0), (0, 2, 3)): 3,
    # no ties
    ((0, 1, 2), (0, 2, 3)): 0,
    ((1, 0, 2), (0, 2, 3)): 2,
    ((1, 2, 0), (0, 2, 3)): 3,
    ((0, 2, 1), (0, 2, 3)): 0,
    ((2, 0, 1), (0, 2, 3)): 2,
    ((2, 1, 0), (0, 2, 3)): 3,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (0, 2, 4)): 18,
    # 2-way ties for low
    ((0, 0, 2), (0, 2, 4)): 10,
    ((0, 2, 0), (0, 2, 4)): 12,
    ((2, 0, 0), (0, 2, 4)): 8,
    # 2-way ties for high
    ((0, 1, 1), (0, 2, 4)): 0,
    ((1, 0, 1), (0, 2, 4)): 2,
    ((1, 1, 0), (0, 2, 4)): 4,
    # no ties
    ((0, 1, 2), (0, 2, 4)): 0,
    ((1, 0, 2), (0, 2, 4)): 2,
    ((1, 2, 0), (0, 2, 4)): 4,
    ((0, 2, 1), (0, 2, 4)): 0,
    ((2, 0, 1), (0, 2, 4)): 2,
    ((2, 1, 0), (0, 2, 4)): 4,
    # from_diag_best_path_type = 19
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (19, 1, 3)): 24,
    # 2-way ties for low
    ((0, 0, 2), (19, 1, 3)): 20,
    ((0, 2, 0), (19, 1, 3)): 22,
    ((2, 0, 0), (19, 1, 3)): 5,
    # 2-way ties for high
    ((0, 1, 1), (19, 1, 3)): 19,
    ((1, 0, 1), (19, 1, 3)): 1,
    ((1, 1, 0), (19, 1, 3)): 3,
    # no ties
    ((0, 1, 2), (19, 1, 3)): 19,
    ((1, 0, 2), (19, 1, 3)): 1,
    ((1, 2, 0), (19, 1, 3)): 3,
    ((0, 2, 1), (19, 1, 3)): 19,
    ((2, 0, 1), (19, 1, 3)): 1,
    ((2, 1, 0), (19, 1, 3)): 3,
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (19, 1, 4)): 25,
    # 2-way ties for low
    ((0, 0, 2), (19, 1, 4)): 20,
    ((0, 2, 0), (19, 1, 4)): 23,
    ((2, 0, 0), (19, 1, 4)): 6,
    # 2-way ties for high
    ((0, 1, 1), (19, 1, 4)): 19,
    ((1, 0, 1), (19, 1, 4)): 1,
    ((1, 1, 0), (19, 1, 4)): 4,
    # no ties
    ((0, 1, 2), (19, 1, 4)): 19,
    ((1, 0, 2), (19, 1, 4)): 1,
    ((1, 2, 0), (19, 1, 4)): 4,
    ((0, 2, 1), (19, 1, 4)): 19,
    ((2, 0, 1), (19, 1, 4)): 1,
    ((2, 1, 0), (19, 1, 4)): 4,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (19, 2, 3)): 26,
    # 2-way ties for low
    ((0, 0, 2), (19, 2, 3)): 21,
    ((0, 2, 0), (19, 2, 3)): 22,
    ((2, 0, 0), (19, 2, 3)): 7,
    # 2-way ties for high
    ((0, 1, 1), (19, 2, 3)): 19,
    ((1, 0, 1), (19, 2, 3)): 2,
    ((1, 1, 0), (19, 2, 3)): 3,
    # no ties
    ((0, 1, 2), (19, 2, 3)): 19,
    ((1, 0, 2), (19, 2, 3)): 2,
    ((1, 2, 0), (19, 2, 3)): 3,
    ((0, 2, 1), (19, 2, 3)): 19,
    ((2, 0, 1), (19, 2, 3)): 2,
    ((2, 1, 0), (19, 2, 3)): 3,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (19, 2, 4)): 27,
    # 2-way ties for low
    ((0, 0, 2), (19, 2, 4)): 21,
    ((0, 2, 0), (19, 2, 4)): 23,
    ((2, 0, 0), (19, 2, 4)): 8,
    # 2-way ties for high
    ((0, 1, 1), (19, 2, 4)): 19,
    ((1, 0, 1), (19, 2, 4)): 2,
    ((1, 1, 0), (19, 2, 4)): 4,
    # no ties
    ((0, 1, 2), (19, 2, 4)): 19,
    ((1, 0, 2), (19, 2, 4)): 2,
    ((1, 2, 0), (19, 2, 4)): 4,
    ((0, 2, 1), (19, 2, 4)): 19,
    ((2, 0, 1), (19, 2, 4)): 2,
    ((2, 1, 0), (19, 2, 4)): 4
}

In [7]:
partial_dp_mat, cur_cell_best_cum_cost, best_paths_mat = ga.warmup_align_2(
    seq_1=seq_1,
    seq_2=seq_2,
    best_paths_mat=best_paths_mat,
    partial_dp_mat=partial_dp_mat,
    gap_open_cost=gap_open_cost,
    cost_mat=cost_mat,
    moves_for_gap_open_penalty_from_left=moves_for_gap_open_penalty_from_left,
    moves_for_gap_open_penalty_from_up=moves_for_gap_open_penalty_from_up,
    situation_mapper=situation_mapper
)

In [10]:
partial_dp_mat, cur_cell_best_cum_cost, best_paths_mat = ga.do_core_align_2(
    seq_1=seq_1,
    seq_2=seq_2,
    best_paths_mat=best_paths_mat,
    partial_dp_mat=partial_dp_mat,
    gap_open_cost=gap_open_cost,
    cost_mat=cost_mat,
    moves_for_gap_open_penalty_from_left=moves_for_gap_open_penalty_from_left,
    moves_for_gap_open_penalty_from_up=moves_for_gap_open_penalty_from_up,
    situation_mapper=situation_mapper
)

In [14]:
cur_cell_best_cum_cost

28

In [15]:
ga.final_cost_to_score(
    cost=cur_cell_best_cum_cost,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
)

-18

In [ ]:
ga.final_score_to_cost(
    score=-23,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
)

In [ ]:
best_paths_mat

In [ ]:
partial_dp_mat

In [13]:
seq_1_aligned, middle_part, seq_2_aligned = ga.traceback_2(
    best_paths_mat=best_paths_mat,
    seq_1=seq_1,
    seq_2=seq_2, 
)

print(seq_1_aligned)
print(middle_part)
print(seq_2_aligned)

GTAGGCGGTC
  **||*| |
--CAGCTG-C


In [ ]:
# seq_1 = "CATGGG"
# seq_1 = "C"
# seq_2 = "ACTG"
# seq_2 = "TATT"
# seq_1 = "ACACAACTAGTGCTACGTAT"
# seq_2 = "T"
# seq_1 = "TC"
# seq_2 = "T"
# seq_1 = "GTCAGCAT"
# seq_2 = "CTCTGAACACG"
# seq_1 = "CGCCTC"
# seq_2 = "GTCG"
# seq_1 = "CGCCT"
# seq_2 = "GTCG"
# seq_1 = "CATGGG"
# seq_2 = "ACTG"
dynamic_prog_num_rows = len(seq_1) + 1
dynamic_prog_num_cols = len(seq_2) + 1

In [ ]:
alignment = ga.align(
    seq_1=seq_1,
    seq_2=seq_2,
    scoring_mat=scoring_mat,
    gap_existence_cost=gap_existence_cost
)
ga.print_alignment(
    *alignment,
    chars_per_line=70
)

In [ ]:
best_paths_mat = [[1, 1, 1, 1, 1], [2, 0, 1, 0, 0], [2, 0, 0, 1, 0], [2, 0, 0, 0, 1], [2, 2, 0, 0, 0], [2, 2, 0, 2, 0], [2, 2, 2, 0, 2]]
best_paths_mat

In [ ]:
alignment = ga.traceback(
    best_paths_mat=best_paths_mat,
    seq_1=seq_1,
    seq_2=seq_2
)
ga.print_alignment(*alignment)